### Implémentation de Exemplar-based inpainting

In [73]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import cv2

### Etape 1a. (Identify the fill front)

In [74]:
# Charger les images
image = Image.open("fleur.tif")
mask_border = Image.open("mask_border.png")
filled_region_mask = Image.open("filled_region_mask.png")
fleurs = Image.open("fleur.tif")

# Convertir l'image en tableau NumPy
mask_border = np.array(mask_border)/255
filled_region_mask = np.array(filled_region_mask)/255
image_np = np.array(image)/255

### Etape 1.b Compute priorities

In [75]:
def is_in_image(i,j,image):
    H,W = image.shape
    return (0<=i<H) and (0<=j<W)

def update_confidence(border_map,confidence_map,filled_region_mask,patch_size):
    delta = patch_size//2
    border_points = np.column_stack(np.where(border_map==1))
    for (y,x) in border_points:
        confidence = 0
        for dx in range(-delta,delta+1):
            for dy in range(-delta,delta+1):
                ny, nx = y+dy, x+dx
                if (is_in_image(ny,nx,border_map)) and filled_region_mask[ny,nx]==1:
                    confidence += confidence_map[ny,nx]
        confidence /= patch_size**2
        confidence_map[y,x] = confidence

def compute_gradient(image):
    gray_image = cv2.cvtColor((image * 255).astype(np.uint8), cv2.COLOR_RGB2GRAY)
    grad_x = cv2.Sobel(gray_image, cv2.CV_64F, 1, 0, ksize=3)
    grad_y = cv2.Sobel(gray_image, cv2.CV_64F, 0, 1, ksize=3)
    return grad_x, grad_y

def compute_normal(x, y, filled_region_mask):
    kernel = np.array([[1, 1, 1],
                       [1, -8, 1],
                       [1, 1, 1]])
    H, W = filled_region_mask.shape[:2]
    x_minus_1 = max(0,x-1)
    x_plus_1 = min(W-1,x+1)
    y_minus_1 = max(0,y-1)
    y_plus_1 = min(H-1,y+1)
    grad = convolve(filled_region_mask.astype(np.float32), kernel, mode='constant')
    nx = grad[y, x_plus_1] - grad[y, x_minus_1]
    ny = grad[y_plus_1, x] - grad[y_minus_1, x]
    norm = np.sqrt(nx**2 + ny**2)
    if norm == 0:
        return 0, 0
    return nx / norm, ny / norm

def compute_data(border_map,image,filled_region_mask):
    border_points = np.column_stack(np.where(border_map==1))
    border_points_with_gradients = []
    grad_x, grad_y = compute_gradient(image)
    for (y,x) in border_points:
        gx = grad_x[y, x]
        gy = grad_y[y, x]
        n_x, n_y = compute_normal(x,y,filled_region_mask)
        data_value = np.abs(gx*n_x+gy*n_y)/255
        border_points_with_gradients.append(((y,x),data_value))
    return border_points_with_gradients

### Etape 2.a Find the patch with maximum priority

In [76]:
def compute_next_pixel_by_priority(border_map,confidence_map,filled_region_mask,image,patch_size):
    update_confidence(border_map,confidence_map,filled_region_mask,patch_size)
    border_points_with_gradients = compute_data(border_map,image,filled_region_mask)
    xmax, ymax = 0, 0
    priority_max = -np.inf
    for ((y, x), data) in border_points_with_gradients:
        priority = confidence_map[y,x] * data
        if priority > priority_max:
            xmax, ymax = x, y
            priority_max = priority
    return (xmax,ymax)

def compute_mask_around_pixel(x,y,filled_region_mask,patch_size):
    mask = np.zeros((patch_size,patch_size,3))
    delta = patch_size//2
    for dx in range(-delta,delta+1):
        for dy in range(-delta,delta+1):
            ny, nx = y+dy, x+dx
            if (is_in_image(ny,nx,filled_region_mask) and filled_region_mask[ny,nx]==1) :
                mask[dy+delta,dx+delta,:] = 1
    return mask

### Etape 2.b Find the exemplar patch that minimises the SSD

In [77]:
image_copy = np.copy(image_np)

def compute_SSD(target_x,target_y,source_patch,mask,image):
    H, W = image.shape[:2]
    patch_size = source_patch.shape[0]
    delta = patch_size//2
    x_min = max(0,target_x-delta)
    x_max = min(W,target_x+delta+1)
    y_min = max(0,target_y-delta)
    y_max = min(H,target_y+delta+1)

    extracted_patch_height = y_max - y_min
    extracted_patch_width = x_max - x_min
    
    target_patch = np.zeros_like(source_patch)
    
    target_patch[0:extracted_patch_height, 0:extracted_patch_width, :] = image[y_min:y_max, x_min:x_max, :]

    diff_all = (target_patch-source_patch)**2
    diff = diff_all * mask
    SSD = np.sum(diff) 
    return SSD
    
def is_outside(i,j,filled_region_mask,patch_size):
    delta = patch_size//2
    H, W = filled_region_mask.shape
    if i - delta < 0 or i + delta >= W or j - delta < 0 or j + delta >= H:
        return False
    considered_patch = filled_region_mask[j-delta:j+delta+1,i-delta:i+delta+1]
    return np.sum(considered_patch)==patch_size**2
    
def find_best_matching_patch(x,y,mask,filled_region_mask,image,patch_size,search_radius=None):
    delta = patch_size//2
    if search_radius is None:
        search_radius = int(image.shape[0] / 2)
    H, W = image.shape[:2]
    # x_min = max(delta,x-search_radius)
    # x_max = min(W-delta,x+search_radius)
    # y_min = max(delta,y-search_radius)
    # y_max = min(H-delta,y+search_radius)
    min_SSD = float('inf')
    optimal_x, optimal_y = 0, 0

    
    for i in range(delta,W-delta):
        for j in range(delta,H-delta):
            if is_outside(i,j,filled_region_mask,patch_size):
                source_patch = image[j-delta:j+delta+1,i-delta:i+delta+1]
                current_SSD = compute_SSD(x,y,source_patch,mask,image)
                if (current_SSD<min_SSD):
                    min_SSD = current_SSD
                    optimal_x = i
                    optimal_y = j
    return optimal_x, optimal_y

### Etape 2.c et 3. Copy image data from Ψqˆ to Ψpˆ + update confidence.

In [78]:
from scipy.ndimage import convolve
def copy_image_data(source_x,source_y,target_x,target_y,mask,filled_region_mask,image,confidence_map):
    patch_size = mask.shape[0]
    delta = patch_size//2
    for i in range(-delta,delta+1):
        for j in range(-delta,delta+1):
            current_source_x, current_source_y = source_x+i, source_y+j
            current_target_x, current_target_y = target_x+i, target_y+j
            if is_in_image(current_target_y,current_target_x,filled_region_mask) and np.all(mask[j+delta,i+delta,:])==0:
                image[current_target_y,current_target_x,:] = image[current_source_y,current_source_x,:]
                filled_region_mask[current_target_y,current_target_x] = 1
                confidence_map[current_target_y,current_target_x] = confidence_map[target_y,target_x]

def update_border(filled_region_mask):
    neighbor_kernel = np.array([[1,1,1],
                          [1,0,1],
                          [1,1,1]])
    not_filled_mask = (1-filled_region_mask).astype(bool)
    has_filled_neighbor = (convolve(filled_region_mask,neighbor_kernel,mode='constant',cval=0)>=1)
    new_border = (not_filled_mask) & (has_filled_neighbor)
    return new_border

### Final implementation

In [ ]:
from tqdm import tqdm
# Charger les images
image = Image.open("image_cercle.png")
mask_border = Image.open("mask_border.png")
filled_region_mask = Image.open("filled_region_mask.png")

# Convertir l'image en tableau NumPy
mask_border = np.array(mask_border)/255
filled_region_mask = np.array(filled_region_mask)/255
image = np.array(image)/255
initial_region_size = np.sum(filled_region_mask == 0)
alpha_channel = image[:,:,3]
image = image[:,:,:3]
image_cielab = cv2.cvtColor((255*image).astype(np.uint8), cv2.COLOR_RGB2Lab)

print(np.sum(filled_region_mask == 0))
confidence_map = np.copy(filled_region_mask)
patch_size = 51

with tqdm(total=initial_region_size) as pbar:
    while np.sum(filled_region_mask == 0) > 0:
        current_x, current_y = compute_next_pixel_by_priority(mask_border,confidence_map,filled_region_mask,image,patch_size)
        mask_patch = compute_mask_around_pixel(current_x,current_y,filled_region_mask,patch_size)
        source_x, source_y = find_best_matching_patch(current_x, current_y,mask_patch,filled_region_mask,image,patch_size)
        copy_image_data(source_x,source_y,current_x,current_y,mask_patch,filled_region_mask,image,confidence_map)
        mask_border = update_border(filled_region_mask)
        #cv2.imwrite("border.png", mask_border * 255)
        #cv2.imwrite("filled.png", filled_region_mask * 255)
        remaining = np.sum(filled_region_mask == 0)
        pbar.update(initial_region_size - remaining)
        initial_region_size = remaining

#cv2.cvtColor(image, cv2.COLOR_Lab2RGB)
image = np.dstack([image, alpha_channel])
# Sauvegarder l'image au format PNG
image = (255 * image).astype(np.uint8)
image = Image.fromarray(image, 'RGBA')

# Sauvegarder l'image au format PNG
image.save("image_cercle_color_inpaint.png")

35713


 18%|██████▌                              | 6383/35713 [00:39<03:27, 141.06it/s]